# CBF Tutorial - Part 2: Dynamic Obstacles

 Authors: Bardh, Tom and Shakiba

## The next example shows how we can use CBFs to navigate around dynamic obstacles.

Again we start with the **nimble_ant** system, which has the following dynamics:
$
\begin{align*}
  \dot{xr} = u = \begin{bmatrix}u_0 \\ u_1\end{bmatrix} \tag{1}\\
\end{align*}
$
Given an initial position $x^{init}$, our goal is to control this system to a goal location $xr^{goal}$.
$
\begin{align*}
xr^{init} = \begin{bmatrix} xr^{init}_0 \\  xr^{init}_1 \end{bmatrix} \qquad	xr^{goal} = \begin{bmatrix} xr^{goal}_0 \\  xr^{goal}_1 \end{bmatrix} \tag{2}\\
\end{align*}
$
However, different from our last example, the obstacles are dynamic. That means that our CBF depends not only on the dynamics of the system, but the obstacles as well. Recall our definition of the Barrier Function:
$
\begin{align}
\dot{B}(x) \ge -\alpha\cdot(B(x))
\end{align}
$
In this case, the barrier function is defined as follows:

$
\begin{align*}
B(xr,xo) = \frac{(xr_0-xo_0)^2}{a^2} + \frac{(xr_1-xo_1)^2}{b^2} - 1
\end{align*}
$

To obtain the derivative, we utilize multivariate chain rule:

$
\begin{align*}
\dot{B}(xr,xo) = \frac{\partial B}{\partial xr} * \dot{xr} + \frac{\partial B}{\partial xo} * \dot{xo}
\end{align*}
$
this is equivalent to
$
\begin{align*}
\dot{B}(xr,xo) = \begin{bmatrix} \frac{2}{a^2} (xr_0 - xo_0) \\  \frac{2}{b^2} (xr_1 - xo_1) \end{bmatrix} * \dot{xr} + 
\begin{bmatrix} \frac{2}{a^2} (xo_0 - xr_0) \\  \frac{2}{b^2} (xo_1 - xr_1) \end{bmatrix} * \dot{xo}
\end{align*}
$
here, $\dot{xr}$ is our control input and lets consider an example where the obstacle moves only left. That is $\dot{xo} = \begin{bmatrix} -1 \\  0 \end{bmatrix}$.
$
\begin{align*}
\dot{B}(xr,xo) = \begin{bmatrix} \frac{2}{a^2} (xr_0 - xo_0) \\  \frac{2}{b^2} (xr_1 - xo_1) \end{bmatrix} * \begin{bmatrix}u_0 \\ u_1\end{bmatrix} +
\begin{bmatrix} \frac{2}{a^2} (xo_0 - xr_0) \\  \frac{2}{b^2} (xo_1 - xr_1) \end{bmatrix} * \begin{bmatrix}-1 \\ 0\end{bmatrix}
\end{align*}
$
To make our implementation easier, we can also represent this in an augmented state
$
\begin{align*}
x = \begin{bmatrix} xr_0 \\  xr_1 \\ xo_0 \\ xo_1 \end{bmatrix}
\end{align*}
$
And in this case
$
\begin{align*}
\dot{B}(x) = \begin{bmatrix} \frac{2}{a^2} (xr_0 - xo_0) \\  \frac{2}{b^2} (xr_1 - xo_1) \\ \frac{2}{a^2} (xo_0 - xr_0) \\  \frac{2}{b^2} (xo_1 - xr_1) \end{bmatrix} * \begin{bmatrix}u_0 \\ u_1 \\ -1 \\ 0\end{bmatrix}
\end{align*}
$
Now, we solve the following quadratic program
$
\begin{align*}
\min_{u} \quad & ||u_{ref} - u||\\
\textrm{s.t.} \quad & \begin{bmatrix} \frac{2}{a^2} (xr_0 - xo_0) \\  \frac{2}{b^2} (xr_1 - xo_1) \\ \frac{2}{a^2} (xo_0 - xr_0) \\  \frac{2}{b^2} (xo_1 - xr_1) \end{bmatrix} * \begin{bmatrix}u_0 \\ u_1 \\ -1 \\ 0\end{bmatrix} u \ge -(\frac{(xr_0-xo_0)^2}{a^2} + \frac{(xr_1-xo_1)^2}{b^2} - 1) \\
\tag{12}
\end{align*}
$
We can pose this a quadratic program.
$
\begin{align*}
\min_{\mathbf{x}} \quad & \frac{1}{2}\mathbf{x}^T P\mathbf{x} + q^T\mathbf{x} \\
\textrm{s.t.} \quad & G\mathbf{x} \le h \tag{13}
\end{align*}
$
In our case, $\mathbf{x} = \dot{x}$ is the decision variable,
Where
$
\begin{align*}
P = I_4 \tag{14}
\end{align*}
$
$
\begin{align*}
q = \begin{bmatrix} -u_{ref0} \\  -u_{ref1} \\ 0 \\ 0 \end{bmatrix} \tag{15}
\end{align*}
$
$
\begin{align*}
G = \begin{bmatrix} \frac{2}{a^2} (xr_0 - xo_0) \\  \frac{2}{b^2} (xr_1 - xo_1) \\ \frac{2}{a^2} (xo_0 - xr_0) \\  \frac{2}{b^2} (xo_1 - xr_1) \end{bmatrix} \tag{16}
\end{align*}
$
$
\begin{align*}
h = (\frac{(xr_0-xo_0)^2}{a^2} + \frac{(xr_1-xo_1)^2}{b^2} - 1) \tag{17}
\end{align*}
$

Lets consider an example where our nimble ant system starts at position $xr = \begin{bmatrix} 0 \\  3.2 \end{bmatrix}$ and an obstacle starts at postion $xo = \begin{bmatrix} xo_0 \\  xo_1 \end{bmatrix} = \begin{bmatrix} 10 \\  3.5 \end{bmatrix}$.


## Code implementation

First, we import the necessary packages for our example.

In [ ]:
%matplotlib ipympl
from sympy import symbols, Matrix, sin, cos, lambdify, exp, sqrt, log, diff, Mul, srepr
from matplotlib.patches import Ellipse
import matplotlib.animation as animation
from cbflib import cbf, cbf_utils, sys_and_ctrl
import cvxopt as cvxopt
import control as control
import matplotlib.pyplot as plt
import numpy as np
from sympy import symbols


In [ ]:
# Robot Goal
x_goal = np.array([10, 3.5])

# The bad set defines the radii of the eclipse. The position is part of the dynamics of the system.
bad_sets = [[3, 2]]

# Parameters for reference controller
ctrl_param = [0.3, 0.3]

# Symbols and equations for the CBF
xr_0, xr_1, xo_0, xo_1, a, b, xr_0_dot, xr_1_dot = symbols(
    'xr_0 xr_1 xo_0 xo_1 a b xr_0_dot xr_1_dot')
symbs = (xr_0, xr_1, xo_0, xo_1, a, b)

# Barrier function - distance of robot to obstacle
B = ((xr_0 - xo_0)/a)**2 + ((xr_1 - xo_1)/b)**2 - 1

# dx = f(x) + g(x)u
f = Matrix([0, 0, -1.0, 0])
g = Matrix([xr_0, xr_1, 0, 0])
degree = 1


In [ ]:
# Initialize CBF
my_CBF = cbf.CBF(B=B, f=f, g=g, states=(
    xr_0, xr_1, xo_0, xo_1), bad_sets=bad_sets, symbs=symbs, degree=1)

# Simulation settings
T_max = 20
n_samples = 100
T = np.linspace(0, T_max, n_samples)
dt = T[1]-T[0]
params = {'x_goal': x_goal, 'bad_sets': bad_sets,
          'ctrl_param': ctrl_param, 'CBF': my_CBF}


In [ ]:
# intial condition
x_0 = np.array([0, 3.2, 10, 3.5])

# Disable cvxopt optimiztaion output
cvxopt.solvers.options['show_progress'] = False
cvxopt.solvers.options['max_iter'] = 1000


In [ ]:
# Simulate system
print('\nComputing trajectories for the initial condition:')
print('x_0\t x_1')
print(x_0[0], '\t', x_0[1])

# Compute output on the nimble ant system for given initial conditions and timesteps T
x = np.zeros((np.size(x_0), len(T)))
x[:, 0] = x_0

# Simulate system
for i in range(len(T)-1):
    x[:, i+1] = x[:, i] + dt * \
        np.array(sys_and_ctrl.nimble_ant_with_agent_f(
            T[i], x[:, i], [], params))

print("\n*Simulation Done. See plot for animation...")


## Code implementation

First, we import the necessary packages for our example.

In [ ]:
# Animate
# Init Plot
fig, ax = plt.subplots()
plt.xlim(0, 12)
plt.ylim(0, 6)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
line1, = ax.plot([], [], lw=2)
line2, = ax.plot([], [], lw=2)

ell = Ellipse((x[2][i-1], x[3][i-1]),
              bad_sets[0][0], bad_sets[0][1], color='g', alpha=0.3)


def init():
    line1.set_data([], [])
    line2.set_data([], [])
    ell.center = (x[2][0], x[3][0])
    return line1, line2


def animate(i):
    line1.set_data((x[0][0:i], x[1][0:i]))
    line2.set_data((x[2][0:i], x[3][0:i]))
    ell.center = (x[2][i], x[3][i])
    return line1, line2, ax.add_patch(ell)


ani = animation.FuncAnimation(
    fig, animate, init_func=init, interval=50, blit=False, frames=n_samples)

plt.show()

print("\n*Animation Complete. Exiting...\n")
